In [348]:
import qiskit
from qiskit.providers.aer import QasmSimulator
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from scipy.optimize import minimize
from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram, plot_gate_map, plot_circuit_layout
from qiskit.tools.monitor import job_monitor
from qiskit.providers.ibmq import least_busy
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager

In [349]:
#configure backend from the to eth architecture
# important to run the current script and see how it works please ensure that you have installed qiskit-terra-installed
configuration = {
    'backend_name': 'qasm_simulator',
    'description': 'OpenSuperQ ETH 7 qubit chip, rev. 1, 12_15702',
    'backend_version': '0.1.0',
    'url': 'http://opensuperq.eu/',
    'sample_name': 'QUDEV_M_12_15702',
    'n_qubits': 7,
    'basis_gates': ['cz', 'rz', 'rx_pi/2'],
    'coupling_map': [[0, 2], [0, 3], [1, 3], [1, 4], [2, 5], [3, 5], [3, 6], [4, 6],[2, 0], [3, 0], [3, 1], [4, 1], [5, 2], [5, 3], [6, 3], [6, 4]],
    # Reduced qubit numbers by 1 compared to 20190823_OSQ_Waypoint1_experimental_parameters.pdf
    'simulator': True,
    'local': True,
    'open_pulse': False,
    'conditional': False,
    'n_registers': 1,  # Really 0, but QISKIT would not allow it, even if 'conditional' is False
    'max_shots': 10000000,
    'memory': True,
    'credits_required': False}

gates =[
    {'name': 'cz',
     'parameters': [],
     'coupling_map':[[0, 2], [0, 3], [1, 3], [1, 4], [2, 5], [3, 5], [3, 6], [4, 6],[2, 0], [3, 0], [3, 1], [4, 1], [5, 2], [5, 3], [6, 3], [6, 4]],
     'qasm_def': 'gate cZ q1,q2 { CZ q1,q2; }'},

    {'name': 'rx_pi/2',
     'parameters': ['matrix'],
     'coupling_map': [[0], [1], [2], [3], [4], [5], [6]],
     'qasm_def': 'unitary(matrix) q1'},

    {'name': 'rz',
     'parameters': ['matrix'],
     'coupling_map': [[0], [1], [2], [3], [4], [5], [6]],
     'qasm_def': 'unitary(matrix) q1'},

    {'name': 'u3_eth',
     'parameters': ['matrix'],
     'coupling_map': [[0], [1], [2], [3], [4], [5], [6]],
     'qasm_def': 'unitary(matrix) q1'}
]


map = [[0, 2], [0, 3], [1, 3], [1, 4], [2, 5], [3, 5], [3, 6], [4, 6],[2, 0], [3, 0], [3, 1], [4, 1], [5, 2], [5, 3], [6, 3], [6, 4]]

In [350]:
#choose qasm backend first
backend = QasmSimulator()

In [351]:
#construct example circuit 
q=QuantumRegister(4)
c=ClassicalRegister(4)
qc=QuantumCircuit(q,c)
qc.h(q[0])
qc.cx(q[2],q[3])
qc.measure(q, c)
print(qc)


                  ┌───┐┌─┐      
q48_0: |0>────────┤ H ├┤M├──────
               ┌─┐└───┘└╥┘      
q48_1: |0>─────┤M├──────╫───────
               └╥┘      ║ ┌─┐   
q48_2: |0>──■───╫───────╫─┤M├───
          ┌─┴─┐ ║       ║ └╥┘┌─┐
q48_3: |0>┤ X ├─╫───────╫──╫─┤M├
          └───┘ ║       ║  ║ └╥┘
 c48_0: 0 ══════╬═══════╩══╬══╬═
                ║          ║  ║ 
 c48_1: 0 ══════╩══════════╬══╬═
                           ║  ║ 
 c48_2: 0 ═════════════════╩══╬═
                              ║ 
 c48_3: 0 ════════════════════╩═
                                


In [352]:
#configure the backend, choose eth basis native gates
backend._configuration.basis_gates = ['cz','rx_pi/2','rz']
transpiledCircuit = transpile(qc, backend=backend, optimization_level=2)

# print the transpiled circuit with respect to eth gates(not included conetivity graph) and optimization level-1
# o-level one will just unroll the circuit to the desired basis gates and taking care of coupling
# no single qubit optimization
print(transpiledCircuit)

             ┌────────────┐┌──────────┐┌────────────┐   ┌─┐                 »
q48_0: |0>───┤ Rx_pi/2(1) ├┤ Rz(pi/2) ├┤ Rx_pi/2(1) ├───┤M├─────────────────»
          ┌─┐└────────────┘└──────────┘└────────────┘   └╥┘                 »
q48_1: |0>┤M├────────────────────────────────────────────╫──────────────────»
          └╥┘                                            ║ ┌─┐              »
q48_2: |0>─╫──────────────────────────────────────────■──╫─┤M├──────────────»
           ║ ┌────────────┐ ┌────────┐ ┌────────────┐ │  ║ └╥┘┌────────────┐»
q48_3: |0>─╫─┤ Rx_pi/2(1) ├─┤ U1(pi) ├─┤ Rx_pi/2(1) ├─■──╫──╫─┤ Rx_pi/2(1) ├»
           ║ └────────────┘ └────────┘ └────────────┘    ║  ║ └────────────┘»
 c48_0: 0 ═╬═════════════════════════════════════════════╩══╬═══════════════»
           ║                                                ║               »
 c48_1: 0 ═╩════════════════════════════════════════════════╬═══════════════»
                                                            ║   

In [353]:
#do the same transpiing as before, but with optimization level 3
#problem of pass manager don't care about basis gates U1 is created
transpiledCircuit = transpile(qc, backend=backend, optimization_level=2)
print(transpiledCircuit)

             ┌────────────┐┌──────────┐┌────────────┐   ┌─┐                 »
q48_0: |0>───┤ Rx_pi/2(1) ├┤ Rz(pi/2) ├┤ Rx_pi/2(1) ├───┤M├─────────────────»
          ┌─┐└────────────┘└──────────┘└────────────┘   └╥┘                 »
q48_1: |0>┤M├────────────────────────────────────────────╫──────────────────»
          └╥┘                                            ║ ┌─┐              »
q48_2: |0>─╫──────────────────────────────────────────■──╫─┤M├──────────────»
           ║ ┌────────────┐ ┌────────┐ ┌────────────┐ │  ║ └╥┘┌────────────┐»
q48_3: |0>─╫─┤ Rx_pi/2(1) ├─┤ U1(pi) ├─┤ Rx_pi/2(1) ├─■──╫──╫─┤ Rx_pi/2(1) ├»
           ║ └────────────┘ └────────┘ └────────────┘    ║  ║ └────────────┘»
 c48_0: 0 ═╬═════════════════════════════════════════════╩══╬═══════════════»
           ║                                                ║               »
 c48_1: 0 ═╩════════════════════════════════════════════════╬═══════════════»
                                                            ║   

In [354]:
#switch back to optimization level 1 and include coupling map
backend.DEFAULT_CONFIGURATION = configuration
transpiledCircuit = transpile(qc, backend=backend, optimization_level=1)
print(transpiledCircuit)

             ┌────────────┐┌──────────┐┌────────────┐   ┌─┐                 »
q48_0: |0>───┤ Rx_pi/2(1) ├┤ Rz(pi/2) ├┤ Rx_pi/2(1) ├───┤M├─────────────────»
          ┌─┐└────────────┘└──────────┘└────────────┘   └╥┘                 »
q48_1: |0>┤M├────────────────────────────────────────────╫──────────────────»
          └╥┘                                            ║ ┌─┐              »
q48_2: |0>─╫──────────────────────────────────────────■──╫─┤M├──────────────»
           ║ ┌────────────┐┌──────────┐┌────────────┐ │  ║ └╥┘┌────────────┐»
q48_3: |0>─╫─┤ Rx_pi/2(1) ├┤ Rz(pi/2) ├┤ Rx_pi/2(1) ├─■──╫──╫─┤ Rx_pi/2(1) ├»
           ║ └────────────┘└──────────┘└────────────┘    ║  ║ └────────────┘»
 c48_0: 0 ═╬═════════════════════════════════════════════╩══╬═══════════════»
           ║                                                ║               »
 c48_1: 0 ═╩════════════════════════════════════════════════╬═══════════════»
                                                            ║   

In [355]:
# transpiler works but not the qasm simulator, its just working with native u1,u2,u3 gates
job = execute(qc,backend=backend,shots=1)

qasm_simulator: simulation failed


In [356]:
#transpile the circuit with respect to optimization level 3
#it fails because most passes and rules are with respect to u1,u2,u3 just optimization level 1,2 and to work
transpiledCircuit = transpile(qc, backend=backend, optimization_level=3)


QiskitError: "Cannot unroll the circuit to the given basis, ['cz', 'rx_pi/2', 'rz']. No rule to expand instruction u3."